In [2]:
import numpy as np

Utilizaremos los siguientes generadores de números uniformes:
- `XORShift (64 bits)`
- `Mersenne Twister (MT19937)`
- `Permuted congruential generator (64-bit, PCG64 DXSM)` 

In [ ]:
# class Xorshift32:
#     def __init__(self, seed):
#         if seed == 0:
#             raise ValueError("Seed must be non-zero")
#         self.state = seed

#     def next(self):
#         x = self.state
#         x ^= (x << 13) & 0xFFFFFFFF
#         x ^= (x >> 17) & 0xFFFFFFFF
#         x ^= (x << 5) & 0xFFFFFFFF
#         self.state = x & 0xFFFFFFFF
#         return self.state


class Xorshift64:
    def __init__(self, seed):
        if seed == 0:
            raise ValueError("Seed must be non-zero")
        self.state = seed

    def next(self):
        x = self.state
        x ^= (x << 13) & 0xFFFFFFFFFFFFFFFF
        x ^= (x >> 7) & 0xFFFFFFFFFFFFFFFF
        x ^= (x << 17) & 0xFFFFFFFFFFFFFFFF
        self.state = x & 0xFFFFFFFFFFFFFFFF
        return self.state


# class Xorshift128:
#     def __init__(self, seed):
#         if len(seed) != 4 or any(s == 0 for s in seed):
#             raise ValueError("Seed must be a list of 4 non-zero uint32 values")
#         self.state = seed

#     def next(self):
#         t = self.state[3]
#         s = self.state[0]
#         self.state[3] = self.state[2]
#         self.state[2] = self.state[1]
#         self.state[1] = s

#         t ^= (t << 11) & 0xFFFFFFFF
#         t ^= (t >> 8) & 0xFFFFFFFF
#         self.state[0] = t ^ s ^ (s >> 19)
#         return self.state[0]


In [15]:
def MT19937(seed, n):
    """
    Mersenne Twister (MT19937) to generate n pseudo-random variables.
    """
    return np.random.Generator(bit_generator=np.random.MT19937(seed)).random(size=n)

In [16]:
def PCG64(seed, n):
    """
    Permuted Congruential Generator (PCG64) to generate n pseudo-random variables.
    """
    return np.random.Generator(bit_generator=np.random.PCG64(seed)).random(size=n)

#### Intensidad del proceso Poisson homogéneo
$$
    \lambda(t) = 30 + 30 \cdot \sin\left(\frac{2\pi t}{24}\right) \text{ (clientes/hora)}
$$

In [10]:
# Intensidad del Poisson proceso homogéneo
def lamda(t):
    """
    Intensity function for a homogeneous Poisson process.
    """
    return 30 + 30 * np.sin((2*np.pi*t)/24)


#### Tiempo de Atención

In [ ]:
def service_time(gen):
    return -np.log(gen) / 40

#### Servidor

In [ ]:
from queuelib.queue import FifoMemoryQueue as Queue    

def server(t_arrivals: list, gen):
    t_server_available = 0
    wait_queue = Queue()
    
    # metrics
    events_queue = []
    t_start_service = []
    t_end_service = []
    
    # process all events
    for t_curr, t_next in zip(t_arrivals, t_arrivals[1:]):
        t_arrival = t_curr
        
        if t_arrival >= t_server_available:
            # server is available
            t_start = t_arrival
            t_service = service_time(gen)
            t_end = t_start + t_service
            
            t_server_available = t_end
            
            # metrics
            t_start_service.append(t_start)
            t_end_service.append(t_end)
            events_queue.append((t_arrival, len(wait_queue)))
            events_queue.append((t_end, len(wait_queue)))

            # process events on queue
            while len(wait_queue) and t_server_available <= t_next:
                enq_event = wait_queue.pop()
                t_start = enq_event
                t_service = service_time(gen)
                t_end = t_start + t_service
            
                t_server_available = t_end
                
                # metrics
                t_start_service.append(t_start)
                t_end_service.append(t_end)
                events_queue.append((t_arrival, len(wait_queue)))
                events_queue.append((t_end, len(wait_queue)))
        else:
            # server is unavailable
            wait_queue.push(t_curr)
            # metrics
            events_queue.append((t_arrival, len(wait_queue)))